# 数据对接天擎

In [10]:
%time
import base64
import json
import os
from io import BytesIO
from math import isnan
import cinrad
import geopandas as gpd
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

# import matplotlib.pyplot as plt
import netCDF4
import numpy as np
# import modin.pandas as pd
import pandas as pd
import shapefile
import xarray as xr
from affine import Affine
from matplotlib.patches import PathPatch
from matplotlib.path import Path
from mpl_toolkits.basemap import Basemap
from ncmaps import Cmaps
from rasterio import features



import xesmf as xe
import redis
from scipy.interpolate import griddata
from scipy.interpolate import interp1d
# import h5netcdf.legacyapi as netCDF4

os.environ["HDF5_USE_FILE_LOCKING"] = 'FALSE'
import datetime as dtt
from datetime import timezone
from datetime import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap

from pylab import *
from matplotlib.font_manager import FontProperties
import pymysql
from pymysql.converters import escape_string
import pickle
import pymssql 
# 查询历史数据的calss

from astropy.convolution import convolve, Gaussian2DKernel, Tophat2DKernel
from astropy.modeling.models import Gaussian2D
## 同步分钟级数据库
# from cma.music.DataQueryClient import DataQueryClient
# ftp
import geojsoncontour
from ftplib import FTP
import os
import datetime as dtt
import shutil
import time


%matplotlib inline

from matplotlib import font_manager

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


In [23]:
# 绘制图片
# 数据加载    
class server_plot():
    def __init__(self,time_hours,city,plot_type,js_status,recv_data):
        self.start = "2023-09-18 19:30:25"
        self.end = "2023-09-20 19:30:25"
        self.time_hours = float(time_hours)
        self.city = city
        self.plot_type = plot_type
        self.js_status = js_status
        self.recv_data = recv_data
        self.max = None
        self.min = None
        #self.rs = redis.Redis(host='127.0.0.1', port=6379,password="tzqxj58660")
        self.conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="051219",db="ZJSZDZDB")
        self.userId = "BEHZ_TZSJ_TZSERVICE" 
        self.pwd = "Liyuan3970!@" 
        self.dataFormat = "json"
        self.rain = None
        self.wind = None
    def data_from_js(self):
        data = pd.read_json(json.dumps(self.recv_data), orient='records')
        self.recv_data = data
        return data
    def wind_from_sql(self):
        start_time = dtt.datetime.strptime(self.start, "%Y-%m-%d %H:%M:%S")
        end_time = dtt.datetime.strptime(self.end, "%Y-%m-%d %H:%M:%S")
        offset = dtt.timedelta(minutes=(-60*8))
        now = (end_time+offset).strftime('%Y-%m-%d %H:%M:%S')
        old = (start_time+offset).strftime('%Y-%m-%d %H:%M:%S')
        sql = """select max(City) as City,max(Cnty) as Cnty, Station_Id_C , max(Province) as Province,max(Station_levl) as Station_levl,
            max(Station_Name) as Station_Name, max(Town) as Town, max(Alti) as Alti, max(Lat) as Lat,max(Lon) as Lon, max(wind) as wind
            from table_station_hour 
            where Datetime between '{start_times}' and '{end_times}' and wind>0 and City='台州市'
            group by Station_Id_C""" 
        rsql = sql.format(start_times=old,end_times=now)
        data = pd.read_sql(rsql, con=self.conn)
        data['WIN_S_Gust_Max'] = data.apply(lambda x: (x.wind - int(str(int(x.wind))[-3:]))/10000, axis = 1)
        data['WIN_D_Gust_Max'] = data.apply(lambda x: int(str(int(x.wind))[-3:]), axis = 1)
        data['value'] = data['WIN_S_Gust_Max']
        self.city_info[self.city]['data']['平均'] = round(data['WIN_S_Gust_Max'].mean(),1)
        self.city_info[self.city]['data']['最大'] = str(round(max(data['WIN_S_Gust_Max']),1))      
        return data
    def temp_from_sql(self):
        start_time = dtt.datetime.strptime(self.start, "%Y-%m-%d %H:%M:%S")
        end_time = dtt.datetime.strptime(self.end, "%Y-%m-%d %H:%M:%S")
        offset = dtt.timedelta(minutes=(-60*8))
        now = (end_time+offset).strftime('%Y%m%d%H%M')+"00"
        old = (start_time+offset).strftime('%Y%m%d%H%M')+"00"
        label = "["+old+","+now+"]"
        client = DataQueryClient(configFile=r"/home/workspace/Data/My_Git/web_met/zdz/common/utils/client.config")
        interfaceId = "statSurfEleInRegion"
        params = {
            'dataCode':"SURF_CHN_MUL_MIN",  #SURF_CHN_MUL_HOR
            'elements':"Cnty,Province,Town,Station_levl,Station_Name,City,Station_Id_C,Lat,Lon,Alti",
            'statEles':'MAX_TEM,MIN_TEM',
            'timeRange':label,
            'adminCodes':"331000",#330000 浙江省
            'eleValueRanges':"TEM:(0,10000)",
            
            'limitCnt':"100000000"
        }
        result = client.callAPI_to_serializedStr(self.userId, self.pwd, interfaceId, params, self.dataFormat)
        result_json = json.loads(result)
        clomns =['Cnty','Province','Town','Station_levl','Station_Name','City','Station_Id_C','Lat','Lon','Alti','tmax','tmin']
        data = pd.DataFrame(result_json['DS'])
        data.columns = clomns
        data = data.astype({'Lat': 'float', 'Lon': 'float','Station_levl':'int','Alti':'float','tmax':'float','tmin':'float'})
        if self.plot_type=="tmax":
            data['value'] = data['tmax']
            self.min = data['value'].min()
            self.max = data['value'].max()
        else:
            data['value'] = data['tmin']
            self.min = data['value'].min()
            self.max = data['value'].max()
        return data
    def rain_from_cloud(self):
        data = pd.read_csv("downfile/rain.csv")
        data['value'] = data['PRE']
        self.rain = data
        return data
    def color_map(self):
        if self.plot_type=="rain":
            hours = self.time_hours
            if hours >12:
                colorslist = ['#FFFFFF','#A3FAFD', '#29D3FD', '#29AAFF', '#2983FF', '#4EAB37', '#46FA35', '#F1F837', '#F1D139', '#F2A932', '#F13237', '#C4343A', '#A43237', '#A632B4', '#D032E1', '#E431FF']# 24降水
                levels = [-1,0.01, 5, 10, 15, 20, 25, 35, 50, 75, 100, 150, 200, 250, 350, 500]
                cmap_nonlin = mpl.colors.ListedColormap(colorslist)  # 自定义颜色映射 color-map
                norm = mpl.colors.BoundaryNorm(levels, cmap_nonlin.N)  # 基于离散区间生成颜色映射索引
            elif hours <=12 and hours >=1:
                colorslist =['#FFFFFF','#A3FAFD', '#29D3FD', '#29AAFF', '#2983FF', '#4EAB37', '#46FA35', '#F1F837', '#F1D139', '#F2A932', '#F13237', '#C4343A', '#A43237', '#A632B4', '#D032E1', '#E431FF']# 06降水
                levels = [-1,3, 4, 5, 10, 15, 20, 25, 35, 50, 60, 70, 80, 90, 100, 110]
                cmap_nonlin = mpl.colors.ListedColormap(colorslist)  # 自定义颜色映射 color-map
                norm = mpl.colors.BoundaryNorm(levels, cmap_nonlin.N)  # 基于离散区间生成颜色映射索引       
            elif hours <1:
                colorslist =['#FFFFFF','#A3FAFD', '#29D3FD', '#29AAFF', '#2983FF', '#4EAB37', '#46FA35', '#F1F837', '#F1D139', '#F2A932', '#F13237', '#C4343A', '#A43237', '#A632B4', '#D032E1', '#E431FF']# 01降水
                levels = [-1,0.01, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 15, 17, 20]
                cmap_nonlin = mpl.colors.ListedColormap(colorslist)  # 自定义颜色映射 color-map
                norm = mpl.colors.BoundaryNorm(levels, cmap_nonlin.N)  # 基于离散区间生成颜色映射索引 
        elif self.plot_type=="wind":
            colorslist = ['#FFFFFF','#CED9FF','#9CFFFF','#FFFF9C','#FFCF9C','#FF9E63','#FF6131','#FF3031','#CE0000']
            levels = [0,0.3,1.6,3.4,5.5,8,10.8,13.9,17.2,28]
            #colorslist = ['#FFFFFF','#CED9FF','#9CFFFF',"#42F217","#FF9E63","#DF16EE","red"]# 风力
            #levels = [0,1.6,3.4,5.5,13.9,17.3,32.6,56]
            cmap_nonlin = mpl.colors.ListedColormap(colorslist)  # 自定义颜色映射 color-map
            norm = mpl.colors.BoundaryNorm(levels, cmap_nonlin.N)  # 基于离散区间生成颜色映射索引 
        elif self.plot_type=="tmax":
            colorslist = ['#0524B7','#092CD2','#0B34F4','#3859F7','#7187F0','#AAB6F3','#C9D1F8','#F8C9CB','#F19599','#F7797D','#F3464D','#F20710','#92080D','#650307']
            level = list(np.linspace(self.min-1, self.max+1, num=14, endpoint=True, retstep=False, dtype=None))
            levels = [round(i,1) for i in level]
            cmap_nonlin = mpl.colors.ListedColormap(colorslist)  # 自定义颜色映射 color-map
            norm = mpl.colors.BoundaryNorm(levels, cmap_nonlin.N)  # 基于离散区间生成颜色映射索引
        elif self.plot_type=="tmin":
            colorslist = ['#FFFFFF','#CED9FF','#9CFFFF','#FFFF9C','#FFCF9C','#FF9E63','#FF6131','#FF3031','#CE0000']
            level = list(np.linspace(self.min-1, self.max+1, num=14, endpoint=True, retstep=False, dtype=None))
            levels = [round(i,1) for i in level]
            cmap_nonlin = mpl.colors.ListedColormap(colorslist)  # 自定义颜色映射 color-map
            norm = mpl.colors.BoundaryNorm(levels, cmap_nonlin.N)  # 基于离散区间生成颜色映射索引  
        return cmap_nonlin,levels
    def decode_xarray(self):
        if self.plot_type=="rain":
            if self.js_status:      
                data = self.data_from_js()
            else:
                data = self.rain_from_cloud()
                self.recv_data = data
        elif self.plot_type=="wind":
            if self.js_status:
                data = self.data_from_js()
            else:
                data = self.wind_from_sql()
                self.recv_data = data
        elif self.plot_type=="tmax" or self.plot_type=="tmin":
            if self.js_status:
                data = self.data_from_js()
            else:
                data = self.temp_from_sql() 
                self.recv_data = data
        lat = np.array(data['Lat'].to_list())
        lon = np.array(data['Lon'].to_list())
        Zi = np.array(data['value'].to_list())
        data_max = max(Zi)
        data_min = min(Zi)
        self.max = data_max
        self.min = data_min
        np.set_printoptions(precision = 2)
        x = np.arange(120.0,122.0,0.01)
        y = np.arange(27.8,29.5,0.01)
        nx0 =len(x)
        ny0 =len(y)
        X, Y = np.meshgrid(x, y)#100*100
        P = np.array([X.flatten(), Y.flatten() ]).transpose()    
        Pi =  np.array([lon, lat ]).transpose()
        Z_linear = griddata(Pi, Zi, P, method = "nearest").reshape([ny0,nx0])
        gauss_kernel = Gaussian2DKernel(0.8)
        smoothed_data_gauss = convolve(Z_linear, gauss_kernel)
        data_xr = xr.DataArray(smoothed_data_gauss, coords=[ y,x], dims=["lat", "lon"])
        return data_xr
    def return_mark(self):
        '''主要返回风场位置'''
        if self.plot_type=="wind":
            mark = self.recv_data
            mark_data = mark[(mark['value']>0)&(mark['City']=="台州市")]
            mark_json = mark_data.to_json(orient='records',force_ascii=False)
        else:
            mark_json = self.recv_data.to_json(orient='records',force_ascii=False)
        return mark_json
    def wind_to_json(self):
        data = pd.read_csv("downfile/min.csv")
        data = data.astype({'Lat': 'float', 'Lon': 'float','PRE': 'float','WIN_S_Gust_Max': 'float', 'WIN_D_Gust_Max': 'float'})
        # data['WIN_S_Gust_Max'] = data['WIN_S_Inst_Max']
        # data['WIN_D_Gust_Max'] = data['WIN_D_INST_Max']
        data['value'] = data['WIN_S_Gust_Max']
        wind = data[(data['value']>15)&(data['value']<100)&(data['City']=="台州市")]
        self.wind = wind
        return wind
    def plot_rain(self):
        wind = self.wind_to_json()
        wind_json = wind.to_json(orient = "records", force_ascii=False)
        data_xr = self.decode_xarray()
        cmaps ,norm = self.color_map()
        lat = data_xr.lat
        lon = data_xr.lon
        lons, lats = np.meshgrid(lon, lat)
        contour = plt.contourf(lons,lats,data_xr,cmap=cmaps,levels = norm)
        geojson = geojsoncontour.contourf_to_geojson(
            contourf=contour,
            ndigits=3,
            unit='mm'
        )
        plt.close()
        return geojson,wind_json    
    def return_geojson(self):
        data_xr = self.decode_xarray()
        # ##########色标和大小#############################
        cmaps ,levels = self.color_map()
        lat = data_xr.lat
        lon = data_xr.lon
        lons, lats = np.meshgrid(lon, lat)
        # contour = plt.contourf(lons,lats,data_xr,cmap=cmaps,levels =levels)
        contour = plt.contourf(lons,lats,data_xr,cmap=cmaps,levels = levels)
        geojson = geojsoncontour.contourf_to_geojson(
            contourf=contour,
            ndigits=3,
            unit='mm'
        )
        plt.close()
        return geojson
    def text_wind_rain(self):
        wind = self.wind
        rain = self.rain
        text = ""
        # rain = pd.read_csv("rian1.csv")
        # wind = pd.read_csv("wind1.csv")
        if self.time_hours ==0.1:   
            text_rain = "目前:"
            rain_tz = rain[rain['City']=="台州市"]
            bins = [0,5,10,100]
            labels = [0,1,2]
            rain_tz['rank']=pd.cut(rain_tz['PRE'],bins,right=False,labels=labels)
            rain_max = rain_tz.sort_values(by="PRE",ascending=False).head(3)
            raintop = rain_max.head(1)
            if raintop['PRE'].values[0]>0: 
                text_rain = text_rain + raintop['Cnty'].values[0] + raintop['Town'].values[0] + "十分钟降水为" + str(raintop['PRE'].values[0]) +"毫米。"
                if len(rain_tz[rain_tz['rank']==1])>0:
                    indextext = "单站雨量较大的有："
                    for index,row in rain_max.iterrows():
                        if row['PRE']>0:
                            indextext = indextext + row['Cnty'] + row['Station_Name'] + str(row['PRE']) + "毫米，"
                    indextext = indextext[:-1] + "。"
                    text_rain = text_rain + indextext
                    if len(rain_tz[rain_tz['rank']==2])>0:
                        text_rain = text_rain + "其中超过10毫米的站有" + str(len(rain_tz[rain_tz['rank']==2])) + "个；"          
            else:
                text_rain = ""
        elif self.time_hours ==1:
            text_rain = "目前:"
            rain_tz = rain[rain['City']=="台州市"]
            bins = [0,10,20,50,80,100,5000]
            labels = [0,1,2,3,4,5]
            rain_tz['rank']=pd.cut(rain_tz['PRE'],bins,right=False,labels=labels)
            rain_max = rain_tz.sort_values(by="PRE",ascending=False).head(3)
            raintop = rain_max.head(1)
            town_max = rain_tz.groupby(['Town','Cnty'])['PRE'].max().sort_values(ascending=False).head(3).to_frame() 
            if raintop['PRE'].values[0]>0: 
                text_rain = text_rain + raintop['Cnty'].values[0] + raintop['Town'].values[0] + "近一小时降水为" + str(raintop['PRE'].values[0]) +"毫米。"
                if len(rain_tz[rain_tz['rank']>=0])>0:
                    indextext = "单站雨量较大的有："
                    for index,row in rain_max.iterrows():
                        if row['PRE']>0:
                            indextext = indextext + row['Cnty'] + row['Station_Name'] + str(row['PRE']) + "毫米，"    
                    indextext = indextext[:-1] + "。"
                    text_rain = text_rain + indextext
                    if len(rain_tz[rain_tz['rank']>1])>0:
                        indextext_town = "单站雨量较大的乡镇有：" 
                        for index,row in town_max.iterrows():
                            if row['PRE']>0:
                                indextext_town = indextext_town + index[1] + index[0] + str(row['PRE']) + "毫米," 
                        indextext_town = indextext_town[:-1] + "。"
                        text_rain = text_rain + indextext_town                            
        elif self.time_hours ==3:
            text_rain = "目前:"
            rain_tz = rain[rain['City']=="台州市"]
            bins = [0,30,50,80,100,5000]
            labels = [0,1,2,3,4]
            rain_tz['rank']=pd.cut(rain_tz['PRE'],bins,right=False,labels=labels)
            rain_max = rain_tz.sort_values(by="PRE",ascending=False).head(3)
            raintop = rain_max.head(1)
            town = rain_tz.groupby(['Town','Cnty'])['PRE'].mean().head(5).sort_values(ascending=False).to_frame() 
            if raintop['PRE'].values[0]>0: 
                text_rain = text_rain + raintop['Cnty'].values[0] + raintop['Town'].values[0] + "近三小时降水为" + str(raintop['PRE'].values[0]) +"毫米。"
                if raintop['PRE'].values[0]>20:
                    indextext_town = "乡镇面雨量较大的有：" 
                    for index,row in town.iterrows():
                        if row['PRE']>0:
                            indextext_town = indextext_town + index[1] + index[0] + str(round(row['PRE'],1)) + "毫米," 
                    indextext_town = indextext_town[:-1] + "。"
                    text_rain = text_rain + indextext_town 
                    if raintop['PRE'].values[0]>50:
                        town_max_text = "其中，出现暴雨及以上的乡镇有："
                        town_max = rain_tz.groupby(['Town','Cnty'])['PRE'].max().sort_values(ascending=False).to_frame()
                        town_nums = town_max[town_max['PRE']>=50]
                        for index,row in town_nums.iterrows():
                            town_max_text = town_max_text + index[1] + index[0] + ","
                        town_max_text = town_max_text[:-1] + "。" 
                        text_rain = text_rain + town_max_text 
        else: 
            rain_tz = rain[rain['City']=="台州市"]
            city_mean = round(rain_tz[rain_tz['PRE']>0]['PRE'].mean(),2)
            bins = [0,30,50,80,100,5000]
            labels = [0,1,2,3,4]
            rain_tz['rank']=pd.cut(rain_tz['PRE'],bins,right=False,labels=labels)
            rain_max = rain_tz.sort_values(by="PRE",ascending=False).head(3)
            raintop = rain_max.head(1)
            average = rain_tz.groupby(['Cnty'])['PRE'].mean().to_frame().sort_values(by="PRE",ascending=False)
            text_rain = "目前:全市面雨量为" + str(city_mean) + "毫米。"
            text_cnty_average = "雨量较大的有："
            for index,row in average.iterrows():
                if row['PRE']>0:
                    text_cnty_average = text_cnty_average + index + str(round(row['PRE'],2)) + "毫米,"  
            text_cnty_average = text_cnty_average[:-1] + "。"
            text_rain = text_rain + text_cnty_average
            if raintop['PRE'].values[0]>0: 
                text_rain = text_rain +"其中，单站累计最大"+ raintop['Cnty'].values[0] + raintop['Town'].values[0] + str(raintop['PRE'].values[0]) +"毫米。"        
        # 风力的统计
        wind = wind[wind['value']>10].sort_values(by="value",ascending=False)
        wind_text = ""
        if len(wind)>0:
            wind_text = wind_text + "全市出现8级以上大风，风力较大的有"
            text_maxwind = ""
            if len(wind)>3:
                wind = wind.head(3)
            else:
                wind = wind
            for index,row in wind.iterrows():
                text_maxwind = text_maxwind + row['Cnty'] + row['Town'] + "-"+ row['Station_Name'] + str(row['value']) +"m/s,"
            text_maxwind = text_maxwind[:-1] + "。"
            wind_text =  wind_text + text_maxwind  
        text = text + text_rain + wind_text
        return text
    def return_province(self):
        data = self.rain_from_cloud()
        lat = np.array(data['Lat'].to_list())
        lon = np.array(data['Lon'].to_list())
        Zi = np.array(data['value'].to_list())
        np.set_printoptions(precision = 2)
        x = np.arange(118.0,123.0,0.01)
        y = np.arange(26,31,0.01)
        nx0 =len(x)
        ny0 =len(y)
        X, Y = np.meshgrid(x, y)#100*100
        P = np.array([X.flatten(), Y.flatten() ]).transpose()    
        Pi =  np.array([lon, lat ]).transpose()
        Z_linear = griddata(Pi, Zi, P, method = "nearest").reshape([ny0,nx0])
        gauss_kernel = Gaussian2DKernel(0.8)
        smoothed_data_gauss = convolve(Z_linear, gauss_kernel)
        data_xr = xr.DataArray(smoothed_data_gauss, coords=[ y,x], dims=["lat", "lon"])
        cmaps ,norm = self.color_map()
        lat = data_xr.lat
        lon = data_xr.lon
        lons, lats = np.meshgrid(lon, lat)
        contour = plt.contourf(lons,lats,data_xr,cmap=cmaps,levels = norm)
        geojson = geojsoncontour.contourf_to_geojson(
            contourf=contour,
            ndigits=3,
            unit='mm'
        )
        plt.close()
        # 风力
        wind = self.wind_to_json()
        wind_json = wind.to_json(orient = "records", force_ascii=False)
        # 文字
        rain = data[data['Province']=="浙江省"]
        rain_max = rain.sort_values(by="PRE",ascending=False).head(3)
        raintop = rain_max.head(1)
        text_rain = ""
        if raintop['PRE'].values[0]>0: 
            text_rain = text_rain +"目前" + raintop['City'].values[0] + raintop['Cnty'].values[0] + raintop['Town'].values[0] +raintop['Station_Name'].values[0] +"出现"+ str(raintop['PRE'].values[0]) +"毫米的降水。"
        average = rain.groupby(['City'])['PRE'].mean().to_frame().sort_values(by="PRE",ascending=False)
        text_cnty_average = "各市面雨量较大的有："
        for index,row in average.iterrows():
            if row['PRE']>0:
                text_cnty_average = text_cnty_average + index + str(round(row['PRE'],2)) + "毫米,"  
        text_cnty_average = text_cnty_average[:-1] + "。"
        text_rain = text_rain + text_cnty_average
        wind = wind[wind['value']>10].sort_values(by="value",ascending=False)
        text = ""
        wind_text = ""
        if len(wind)>0:
            wind_text = wind_text + "全省出现8级以上大风，风力较大的有"
            text_maxwind = ""
            if len(wind)>3:
                wind = wind.head(3)
            else:
                wind = wind
            for index,row in wind.iterrows():
                text_maxwind = text_maxwind + row['Cnty'] + row['Town'] + "-"+ row['Station_Name'] + str(row['value']) +"m/s,"
            text_maxwind = text_maxwind[:-1] + "。"
            wind_text =  wind_text + text_maxwind  
        text = text + text_rain + wind_text
        return geojson,text,wind_json
time_hours,city,plot_type,js_status,recv_data = 1,'taizhou','rain',False,None
worker = server_plot(time_hours,city,plot_type,js_status,recv_data)
worker.return_province()

'目前台州市玉环市龙溪乡花岩浦出现91.7毫米的降水。各市面雨量较大的有：台州市9.54毫米。全省出现8级以上大风，风力较大的有椒江区大陈镇-一江山15.7m/s,椒江区大陈镇-一江山15.4m/s。'

In [ ]:
%time
import base64
import json
import os
from io import BytesIO
from math import isnan
import cinrad
import geopandas as gpd
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

# import matplotlib.pyplot as plt
import netCDF4
import numpy as np
# import modin.pandas as pd
import pandas as pd
import shapefile
import xarray as xr
from affine import Affine
from matplotlib.patches import PathPatch
from matplotlib.path import Path
from mpl_toolkits.basemap import Basemap
from ncmaps import Cmaps
from rasterio import features



import xesmf as xe
import redis
from scipy.interpolate import griddata
from scipy.interpolate import interp1d
# import h5netcdf.legacyapi as netCDF4

os.environ["HDF5_USE_FILE_LOCKING"] = 'FALSE'
import datetime as dtt
from datetime import timezone
from datetime import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap

from pylab import *
from matplotlib.font_manager import FontProperties
import pymysql
from pymysql.converters import escape_string
import pickle
import pymssql 
# 查询历史数据的calss

from astropy.convolution import convolve, Gaussian2DKernel, Tophat2DKernel
from astropy.modeling.models import Gaussian2D
## 同步分钟级数据库
from cma.music.DataQueryClient import DataQueryClient
# ftp
import geojsoncontour
from ftplib import FTP
import os
import datetime as dtt
import shutil
import time


%matplotlib inline

from matplotlib import font_manager
# 自定义画图类
class nlcmap(LinearSegmentedColormap):
    """A nonlinear colormap"""

    name = 'nlcmap'

    def __init__(self, cmap, levels):
        self.cmap = cmap
        self.monochrome = self.cmap.monochrome
        self.levels = np.asarray(levels, dtype='float64')
        self._x = self.levels/ self.levels.max()
        self.levmax = self.levels.max()
        self.levmin = self.levels.min()
        self._y = np.linspace(self.levmin, self.levmax, len(self.levels))

    def __call__(self, xi, alpha=1.0, **kw):
        yi = np.interp(xi, self._x, self._y)
        return self.cmap(yi/self.levmax, alpha)

class server_plot():
    def __init__(self,start,end,city,plot_type):
        self.start = start 
        self.end = end
        self.city = city
        self.plot_type = plot_type
        self.rs = redis.Redis(host='127.0.0.1', port=6379,password="tzqxj58660")
        self.conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="tzqxj58660",db="ZJSZDZDB")
        self.userId = "BEHZ_TZSJ_TZSERVICE" 
        self.pwd = "Liyuan3970!@" 
        self.dataFormat = "json"
        self.shp_path = "/home/workspace/Data/My_Git/web_met/static/data/shpfile/xian/"
        self.max = None
        self.min = None
        self.city_info = {
            "taizhou":{
                "static":r"""<table class="layui-table" style="width: 90%;margin-left: 5%;margin-right: 5%;font-size: 5px;">
                    <thead>
                      <tr>
                        <th>  </th>
                        <th>{0[0]}</th>
                        <th>{0[1]}</th>
                        <th>{0[2]}</th>
                        <th>{0[3]}</th>
                        <th>{0[4]}</th>
                        <th>{0[5]}</th>
                        <th>{0[6]}</th>
                        <th>{0[7]}</th>
                        <th>{0[8]}</th>
                        <th>{0[9]}</th>
                        <th>{0[10]}</th>
                      </tr> 
                    </thead>
                    <tbody>
                      <tr>
                        <td>雨量</td>
                        <td>{1[0]}</td>
                        <td>{1[1]}</td>
                        <td>{1[2]}</td>
                        <td>{1[3]}</td>
                        <td>{1[4]}</td>
                        <td>{1[5]}</td>
                        <td>{1[6]}</td>
                        <td>{1[7]}</td>
                        <td>{1[8]}</td>
                        <td>{1[9]}</td>
                        <td>{1[10]}</td>
                      </tr>
                    </tbody>
                  </table>""",
                "data":{
                    "三门":[28.99,121.41],
                    "临海":[28.82,121.15],
                    "仙居":[28.73,120.62],
                    "天台":[29.15,120.97],
                    "椒江":[28.66,121.42],
                    "温岭":[28.38,121.35],
                    "路桥":[28.54,121.41],
                    "黄岩":[28.59,121.04], 
                    "玉环":[28.15,121.23],
                    "平均":None,
                    "最大":None}       
                    },
                "wind":None          
        }
        
    def city_shp(self,data_xr):
        shp_da = self.add_shape_coord_from_data_array(data_xr,self.shp_path+"xian.shp", "country")
        awash_da = shp_da.where(shp_da.country<9, other=np.nan)
        return awash_da
    def transform_from_latlon(self,lat, lon):
        lat = np.asarray(lat)
        lon = np.asarray(lon)
        trans = Affine.translation(lon[0], lat[0])
        scale = Affine.scale(lon[1] - lon[0], lat[1] - lat[0])
        return trans * scale    
    def rasterize(self,shapes, coords, latitude='lat', longitude='lon',fill=np.nan, **kwargs):
        transform = self.transform_from_latlon(coords[latitude], coords[longitude])
        out_shape = (len(coords[latitude]), len(coords[longitude]))
        raster = features.rasterize(shapes, out_shape=out_shape,
                                fill=fill, transform=transform,
                                dtype=float, **kwargs)
        spatial_coords = {latitude: coords[latitude], longitude: coords[longitude]}
        return xr.DataArray(raster, coords=spatial_coords, dims=(latitude, longitude))
    def add_shape_coord_from_data_array(self,xr_da, shp_path, coord_name):   
        shp_gpd = gpd.read_file(shp_path)
        shapes = [(shape, n) for n, shape in enumerate(shp_gpd.geometry)]
        xr_da[coord_name] = self.rasterize(shapes, xr_da.coords, longitude='lon', latitude='lat')
        return xr_da
    def basemask(self,cs, ax, map, shpfile):
        sf = shapefile.Reader(shpfile)
        vertices = []
        codes = []
        for shape_rec in sf.shapeRecords():
            if shape_rec.record[0]:  
                pts = shape_rec.shape.points
                prt = list(shape_rec.shape.parts) + [len(pts)]
                for i in range(len(prt) - 1):
                    for j in range(prt[i], prt[i+1]):
                        vertices.append(map(pts[j][0], pts[j][1]))
                    codes += [Path.MOVETO]
                    codes += [Path.LINETO] * (prt[i+1] - prt[i] -2)
                    codes += [Path.CLOSEPOLY]
                clip = Path(vertices, codes)
                clip = PathPatch(clip, transform = ax.transData)    
        for contour in cs.collections:
            contour.set_clip_path(clip)
    def color_map(self):
        if self.plot_type=="rain":
            start_time = dtt.datetime.strptime(self.start, "%Y-%m-%d %H:%M:%S")
            end_time = dtt.datetime.strptime(self.end, "%Y-%m-%d %H:%M:%S")
            hours = (end_time-start_time).total_seconds()//3600
            if hours >12:
                colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 24降水
                levels = [0,1,10,25,50,100,250,1000]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels)
            elif hours <=12 and hours >6:
                colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 12降水
                levels = [0,1,5,15,30,70,140,250]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels)
            elif hours <=6 and hours >3:
                colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 06降水
                levels = [0,1,4,13,25,60,120,250]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels)
            elif hours <=3 and hours >1:
                colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 03降水
                levels = [0,1,3,10,20,50,70,150]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels)
            elif hours <=1:
                colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 01降水
                levels = [0,1,2,7,15,40,50,100]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels) 
        elif self.plot_type=="wind":
            colorslist = ['#FFFFFF','#CED9FF','#9CFFFF',"#42F217","#FF9E63","#DF16EE","red"]# 风力
            levels = [0,1.6,3.4,5.5,13.9,17.3,32.6,56]
            cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
            cmap_nonlin = nlcmap(cmaps, levels) 
        elif self.plot_type=="tmax":
            level = list(np.linspace(self.min-1, self.max+1, num=14, endpoint=True, retstep=False, dtype=None))
            levels = [round(i,1) for i in level]
            cmap_nonlin = 'Reds'
        elif self.plot_type=="tmin":
            level = list(np.linspace(self.min-1, self.max+1, num=14, endpoint=True, retstep=False, dtype=None))
            levels = [round(i,1) for i in level]
            cmap_nonlin = 'Blues_r'      
        return cmap_nonlin,levels
    def get_rain_data(self):
        start_time = dtt.datetime.strptime(self.start, "%Y-%m-%d %H:%M:%S")
        end_time = dtt.datetime.strptime(self.end, "%Y-%m-%d %H:%M:%S")
        offset = dtt.timedelta(minutes=(-60*8))
        now = (end_time+offset).strftime('%Y%m%d%H%M')+"00"
        old = (start_time+offset).strftime('%Y%m%d%H%M')+"00"
        label = "["+old+","+now+"]"
        client = DataQueryClient(configFile=r"/home/workspace/Data/My_Git/web_met/zdz/common/utils/client.config")
        interfaceId = "statSurfEleInRegion"
        params = {
            'dataCode':"SURF_CHN_MUL_MIN",  #SURF_CHN_MUL_HOR
            'elements':"Cnty,Province,Town,Station_levl,Station_Name,City,Station_Id_C,Lat,Lon,Alti",
            'statEles':'SUM_PRE',
            'timeRange':label,
            'adminCodes':"331000",#330000 浙江省
            "statEleValueRanges":"SUM_PRE:(0,10000)",
            'limitCnt':"100000000"
        }
        result = client.callAPI_to_serializedStr(self.userId, self.pwd, interfaceId, params, self.dataFormat)
        result_json = json.loads(result)
        clomns =['Cnty','Province','Town','Station_levl','Station_Name','City','Station_Id_C','Lat','Lon','Alti','PRE']
        data = pd.DataFrame(result_json['DS'])
        data.columns = clomns
        data = data.astype({'Lat': 'float', 'Lon': 'float','Station_levl':'int','PRE': 'float','Alti':'float'})
        data['value'] = data['PRE']
        rain_mean = data.groupby(['Cnty'])['value'].mean().to_dict()
        rain = sorted(rain_mean.items(),key = lambda x:x[1],reverse = True)
        key_list = []
        value_list = []
        for i in range(len(rain)):
            key = rain[i][0][0:2]
            value = round(rain[i][1],1)
            key_list.append(key)
            value_list.append(str(value))
            if key in self.city_info[self.city]['data'].keys():
                self.city_info[self.city]['data'][key].append(value)
        key_list.append("最大")
        key_list.append("平均")
        value_list.append(str(round(max(data['PRE']),1)))
        value_list.append(str(round(data['PRE'].mean(),1)))
        totle = self.city_info[self.city]['static']
        replace = totle.format(key_list, value_list)
        self.city_info[self.city]['static'] = replace
        self.city_info[self.city]['data']['平均'] = round(data['PRE'].mean(),1)
        self.city_info[self.city]['data']['最大'] = str(round(max(data['PRE']),1))
        return data
    def get_wind_data(self):
        start_time = dtt.datetime.strptime(self.start, "%Y-%m-%d %H:%M:%S")
        end_time = dtt.datetime.strptime(self.end, "%Y-%m-%d %H:%M:%S")
        offset = dtt.timedelta(minutes=(-60*8))
        now = (end_time+offset).strftime('%Y-%m-%d %H:%M:%S')
        old = (start_time+offset).strftime('%Y-%m-%d %H:%M:%S')
        sql = """select max(City) as City,max(Cnty) as Cnty, Station_Id_C , max(Province) as Province,max(Station_levl) as Station_levl,
            max(Station_Name) as Station_Name, max(Town) as Town, max(Alti) as Alti, max(Lat) as Lat,max(Lon) as Lon, max(wind) as wind
            from table_station_hour 
            where Datetime between '{start_times}' and '{end_times}' and wind>0 and City='台州市'
            group by Station_Id_C""" 
        rsql = sql.format(start_times=old,end_times=now)
        data = pd.read_sql(rsql, con=self.conn)
        data['WIN_S_Gust_Max'] = data.apply(lambda x: (x.wind - int(str(int(x.wind))[-3:]))/10000, axis = 1)
        data['WIN_D_Gust_Max'] = data.apply(lambda x: int(str(int(x.wind))[-3:]), axis = 1)
        data['value'] = data['WIN_S_Gust_Max']
        self.city_info[self.city]['data']['平均'] = round(data['WIN_S_Gust_Max'].mean(),1)
        self.city_info[self.city]['data']['最大'] = str(round(max(data['WIN_S_Gust_Max']),1))      
        return data
    def get_temp_data(self):
        start_time = dtt.datetime.strptime(self.start, "%Y-%m-%d %H:%M:%S")
        end_time = dtt.datetime.strptime(self.end, "%Y-%m-%d %H:%M:%S")
        offset = dtt.timedelta(minutes=(-60*8))
        now = (end_time+offset).strftime('%Y%m%d%H%M')+"00"
        old = (start_time+offset).strftime('%Y%m%d%H%M')+"00"
        label = "["+old+","+now+"]"
        client = DataQueryClient(configFile=r"/home/workspace/Data/My_Git/web_met/zdz/common/utils/client.config")
        interfaceId = "statSurfEleInRegion"
        params = {
            'dataCode':"SURF_CHN_MUL_MIN",  #SURF_CHN_MUL_HOR
            'elements':"Cnty,Province,Town,Station_levl,Station_Name,City,Station_Id_C,Lat,Lon,Alti",
            'statEles':'MAX_TEM,MIN_TEM',
            'timeRange':label,
            'adminCodes':"331000",#330000 浙江省
            'eleValueRanges':"TEM:(0,10000)",
            
            'limitCnt':"100000000"
        }
        result = client.callAPI_to_serializedStr(self.userId, self.pwd, interfaceId, params, self.dataFormat)
        result_json = json.loads(result)
        clomns =['Cnty','Province','Town','Station_levl','Station_Name','City','Station_Id_C','Lat','Lon','Alti','tmax','tmin']
        data = pd.DataFrame(result_json['DS'])
        data.columns = clomns
        data = data.astype({'Lat': 'float', 'Lon': 'float','Station_levl':'int','Alti':'float','tmax':'float','tmin':'float'})
        if self.plot_type=="tmax":
            data['value'] = data['tmax']
            self.min = data['value'].min()
            self.max = data['value'].max()
        else:
            data['value'] = data['tmin']
            self.min = data['value'].min()
            self.max = data['value'].max()
        return data
    def decode_uv(self,wspd,wdir):
        deg = 180.0/np.pi
        rad = np.pi/180.0
        u = []
        v = []
        for i in range(len(wspd)):
            u.append(-wspd[i]*np.sin(wdir[i]*rad))
            v.append(-wspd[i]*np.cos(wdir[i]*rad))
        u_np = np.array(u)
        v_np = np.array(v)
        return u_np,v_np
    def decode_xarray(self):
        if self.plot_type=="rain":
            data = self.get_rain_data()
        elif self.plot_type=="wind":
            data = self.get_wind_data()
            self.city_info[self.city]['wind'] = data[data['WIN_S_Gust_Max']>13.8] 
        elif self.plot_type=="tmax" or self.plot_type=="tmin":
            data = self.get_temp_data() 
        lat = np.array(data['Lat'].to_list())
        lon = np.array(data['Lon'].to_list())
        Zi = np.array(data['value'].to_list())
        data_max = max(Zi)
        data_min = min(Zi)
        np.set_printoptions(precision = 2)
        x = np.arange(120.0,122.0,0.05)
        y = np.arange(27.8,29.5,0.05)
        nx0 =len(x)
        ny0 =len(y)
        X, Y = np.meshgrid(x, y)#100*100
        P = np.array([X.flatten(), Y.flatten() ]).transpose()    
        Pi =  np.array([lon, lat ]).transpose()
        Z_linear = griddata(Pi, Zi, P, method = "nearest").reshape([ny0,nx0])
        gauss_kernel = Gaussian2DKernel(0.8)
        smoothed_data_gauss = convolve(Z_linear, gauss_kernel)
        data_xr = xr.DataArray(smoothed_data_gauss, coords=[ y,x], dims=["lat", "lon"])
        return data_xr
    def plot_mark(self,ax,m,awash_da):
        font = font_manager.FontProperties(fname="/home/workspace/Data/My_Git/web_met/static/data/simkai.ttf")
        if self.plot_type=="rain":
            data = self.city_info[self.city]['data']
            for key, value in data.items():
                if type(value)==list:
                    ax.text(value[1], value[0], str(value[2]),fontsize=10,fontproperties=font, color='black',bbox=dict(facecolor='yellow', edgecolor='red'),alpha=0.5)
                    ax.text(value[1]+0.008, value[0]+0.05, key,fontsize=10, weight=5,color='black',alpha=1,fontproperties=font)
        elif self.plot_type=="wind":
            x = np.array(self.city_info[self.city]['wind']['Lon'].to_list())
            y = np.array(self.city_info[self.city]['wind']['Lat'].to_list())
            wspd = np.array(self.city_info[self.city]['wind']['WIN_S_Gust_Max'].to_list())
            wdir = np.array(self.city_info[self.city]['wind']['WIN_D_Gust_Max'].to_list())
            u,v = self.decode_uv(wspd,wdir)
            ax.barbs(x,y,u,v, color='red') 
        elif self.plot_type=="tmin" or self.plot_type=="tmax": 
            len_lat = len(awash_da.lat.data)
            len_lon = len(awash_da.lon.data)
            for i in range(len_lon-1):
                for j in range(len_lat-1):
                    y0 = round(27.8+j*0.05,2)
                    x0 = round(120.0+i*0.05,2)
                    if not isnan(awash_da.data[j,i]):
                        ax.text(x0,y0,str(int(awash_da.data[j,i])),fontsize= 7,fontweight = 800 ,color ="black")
    def plot_img(self):
        data_xr = self.decode_xarray()
        # ##########色标和大小#############################
        cmaps ,levels = self.color_map()
        fig = plt.figure(figsize=[10,10]) 
        ax = fig.add_subplot(111)
        awash_da = self.city_shp(data_xr)
        lat = data_xr.lat
        lon = data_xr.lon
        m = Basemap(llcrnrlon=120.2,
            llcrnrlat=27.8,
            urcrnrlon=122,
            urcrnrlat=29.5,
            resolution = None, 
            projection = 'cyl')
        lons, lats = np.meshgrid(lon, lat)
        cs =m.contourf(lons,lats,data_xr,ax=ax, cmap=cmaps,levels =levels)
        m.readshapefile(self.shp_path + 'xian','xian',color='k',linewidth=1.2)
        plt.axis('off')  
        # 在图上绘制色标
        rect1 = [0.35, 0.25, 0.03, 0.12]        
        ax2 = plt.axes(rect1,frameon='False')
        ax2.set_xticks([])
        ax2.set_yticks([])
        ax2.spines['top'].set_visible(False)
        ax2.spines['bottom'].set_visible(False)
        ax2.spines['left'].set_visible(False)
        ax2.spines['right'].set_visible(False)
        m.colorbar(cs, location='right', size='30%', pad="-100%",ax = ax2)       
        self.basemask(cs, ax, m, self.shp_path+'xian') 
        # 绘制mark
        self.plot_mark(ax,m,awash_da)    
        buffer = BytesIO()
        plt.savefig(buffer,bbox_inches='tight')  
        plot_img = buffer.getvalue()
        imb = base64.b64encode(plot_img) 
        ims = imb.decode()
        imd = "data:image/png;base64,"+ims
#         plt.close()
        return imd


plot_type = "rain"
city = "taizhou"
start = '2023-08-24 20:00:00'
end = '2023-08-25 20:00:00' 
worker = server_plot(start,end,city,plot_type)
# data = worker.plot_rain()
data = worker.plot_img()

# plot_type = "wind"
# worker = server_plot(start,end,city,plot_type)
# worker.get_wind_data()


# plot_type = "tmin"
# worker = server_plot(start,end,city,plot_type)
# # data = worker.get_temp_data()
# img = worker.plot_img()



# 实时数据的对接

In [55]:
%%time
import pymysql
import pymssql 
import numpy as np
import pandas as pd
import pickle
import redis
import datetime as dtt
import json
from cma.music.DataQueryClient import DataQueryClient
class station_zdz():
    def __init__(self):
        #self.rs = redis.Redis(host='127.0.0.1', port=6379)
        self.conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="tzqxj58660",db="ZJSZDZDB")
        self.userId = "BEHZ_TZSJ_TZSERVICE" 
        self.pwd = "Liyuan3970!@" 
        self.dataFormat = "json"
    def sql_now(self,value_index,boundary):
        utc_now = dtt.datetime.utcnow()
        offset = dtt.timedelta(seconds=-60)
        label = (utc_now+ offset).strftime('%Y%m%d%H%M') + "00"
        client = DataQueryClient(configFile=r"/home/workspace/Data/My_Git/web_met/zdz/common/utils/client.config")
        interfaceId = "getSurfEleInRectByTime"
        params = {
            'dataCode':"SURF_CHN_MUL_MIN",  #SURF_CHN_MUL_HOR
            'elements':"Datetime,Station_Id_C,Station_Name,Province,City,Cnty,Town,Station_levl,Station_Type,Lat,Lon,Alti,PRE_1h,TEM,VIS_HOR_1MI,WIN_S_Gust_Max,WIN_D_Gust_Max",
            'times':label,
            'minLat':str(boundary[0]),
            'minLon':str(boundary[2]),
            'maxLat':str(boundary[1]),
            'maxLon':str(boundary[3]),
            'limitCnt':"100000000"
        }
        result = client.callAPI_to_serializedStr(self.userId, self.pwd, interfaceId, params, self.dataFormat)
        result_json = json.loads(result)
        clomns = params['elements'].split(",")
        data = pd.DataFrame(result_json['DS'],columns=clomns)
        data.columns =  ["TTime","IIIII","StationName","Province","City","County","Town","ZoomLevel","Type","lat","lon","Height","Ri","T","V","fFy","dFy"]
        data = data.astype({'lat': 'float', 'lon': 'float', 'ZoomLevel': 'int', 'Ri': 'float', 'T': 'float', 'V': 'float', 'fFy': 'float', 'dFy': 'float'})
        data["Ri"]=data["Ri"].round(2)
        data["T"]=data["T"].round(2)
        data["V"]=data["V"].round(2)
        data["fFy"]=data["fFy"].round(2)
        data["dFy"]=data["dFy"].round(2)
        if value_index==0:
            data = data[(data['Ri']<5000) & (data['Ri']>0)]
            data['value'] = data['Ri']
            data['value'] = data['value']*10          
        elif value_index==1:
            data = data[data['T']<5000]
            data['value'] = data['T']
            data['value'] = data['value']*10
        elif value_index==2:
            data = data[data['T']<5000]
            data['value'] = data['T'] 
            data['value'] = data['value']*10
        elif value_index==3:
            data = data[data['fFy']<5000]
            data['value'] = data['fFy']
            data['value'] = data['value']*10
        elif value_index==4:
            data = data[data['V']<30000]
            data['value'] = data['V'] 
        return data

worker = station_zdz()
value_index = 0
boundary = [25,35,115,125]
worker.sql_now(value_index,boundary)   


URL: http://10.135.90.120:80/music-ws/api?serviceNodeId=NMIC_MUSIC_CMADAAS&method=callAPI_to_serializedStr&userId=BEHZ_TZSJ_TZSERVICE&interfaceId=getSurfEleInRectByTime&language=Python&clientversion=V2.0.0&dataCode=SURF_CHN_MUL_MIN&elements=Datetime,Station_Id_C,Station_Name,Province,City,Cnty,Town,Station_levl,Station_Type,Lat,Lon,Alti,PRE_1h,TEM,VIS_HOR_1MI,WIN_S_Gust_Max,WIN_D_Gust_Max&times=20230717025300&minLat=25&minLon=115&maxLat=35&maxLon=125&limitCnt=100000000&dataFormat=json&timestamp=1689562451850&nonce=35303d44-244d-11ee-8497-a0369f70ad47&sign=D9CE546EB99E5FD8BA8A9A46B13EC7FF
CPU times: user 147 ms, sys: 8 ms, total: 155 ms
Wall time: 558 ms


,TTime,IIIII,StationName,Province,City,County,Town,ZoomLevel,Type,lat,lon,Height,Ri,T,V,fFy,dFy,value
14,2023-07-17 02:53:00,K4848,钟林山,浙江省,绍兴市,新昌县,羽林街道,14,999999,29.4833,120.9622,70,6.0,29.5,999998.0,2.4,305.0,60.0
19,2023-07-17 02:53:00,K6080,大盘,浙江省,金华市,磐安县,大盘镇,14,999999,29.0053,120.5611,544,0.1,25.7,999998.0,0.0,122.0,1.0
22,2023-07-17 02:53:00,K2848,岭口,浙江省,宁波市,宁海县,西店镇,14,999999,29.4894,121.3675,166,0.5,26.9,999998.0,2.9,255.0,5.0
37,2023-07-17 02:53:00,K8801,亭旁,浙江省,台州市,三门县,亭旁镇,14,999999,29.0436,121.3403,108,0.1,28.9,999998.0,999999.0,999999.0,1.0
40,2023-07-17 02:53:00,K5611,周家浜,浙江省,嘉兴市,桐乡市,洲泉镇,14,999999,30.5625,120.3417,3,0.1,29.6,999998.0,0.4,239.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8858,2023-07-17 02:53:00,K4851,猪山水厂,浙江省,绍兴市,新昌县,南明街道,14,999999,29.4897,120.9072,90,0.3,29.5,999998.0,999998.0,999998.0,3.0
8860,2023-07-17 02:53:00,K2728,袁夹岙,浙江省,宁波市,奉化区,萧王庙街道,14,999999,29.6278,121.3028,181,1.9,27.8,999998.0,3.3,43.0,19.0
8868,2023-07-17 02:53:00,K4853,西花园,浙江省,绍兴市,新昌县,澄潭街道,14,999999,29.4500,120.7833,50,0.2,30.8,999998.0,1.4,21.0,2.0
8871,2023-07-17 02:53:00,K4852,南岩社区,浙江省,绍兴市,新昌县,七星街道,14,999999,29.5147,120.8328,100,0.9,28.7,999998.0,2.9,72.0,9.0


# 时段统计

In [ ]:
    def comput_city(self,city_code,start,end):
        """快报或者统计数据的接口"""
        # getSurfEleInRegionByTime
        date_start = dtt.datetime.strptime(start,'%Y-%m-%d %H:%M:%S') 
        date_end = dtt.datetime.strptime(end,'%Y-%m-%d %H:%M:%S') 
        offset = dtt.timedelta(minutes=-60*8)
        label_start = (date_start + offset).strftime('%Y%m%d%H%M') + "00"
        label_end = (date_end + offset).strftime('%Y%m%d%H%M') + "00"   
        labels = "[" + label_start + "," + label_end + "]"
        #timelabel = self.decode_time_str(timesdelay)
        client = DataQueryClient()
        interfaceId = "statSurfEleInRegion"  
        params = {
            'dataCode':"SURF_CHN_MUL_MIN",  #SURF_CHN_MUL_HOR
            'adminCodes':"331000",
            'elements':"Province,City,Cnty,Town,Lat,Lon,Alti,Station_levl,Station_Name,Station_Id_C,",
            'timeRange':labels,
            'statEles':'MAX_TEM,MIN_TEM,SUM_PRE,MAX_PRE_1h,MIN_VIS_HOR_1MI,MAX_WIN_S_Gust_Max,MAX_WIN_S_Avg_2mi',
            #'orderBy':"MAX_WIN_S_Inst_Max:desc",
            'staLevels':"011,012,013,014,015,016", # 12国家站 14区域站
            'limitCnt':"100000000"
        }
        result = client.callAPI_to_serializedStr(self.userId, self.pwd, interfaceId, params, self.dataFormat)
        #rint(result)
        result_json = json.loads(result)
        clomns = "Province,City,Cnty,Town,Lat,Lon,Alti,Station_levl,Station_Name,Station_Id_C,MAX_TEM,MIN_TEM,SUM_PRE,MAX_PRE_1h,MIN_VIS_HOR_1MI,MAX_WIN_S_Gust_Max,MAX_WIN_S_Avg_2mi".split(",")
        data = pd.DataFrame(result_json['DS'],columns=clomns)
        data.columns = "Province,City,Cnty,Town,Lat,Lon,Height,Stationlevl,StationName,IIIII,tmax,tmin,rain,rainhour,view,wind,windave".split(",")
        return data

# 降水统计

In [42]:
%%time
import pymysql
import pymssql 
import numpy as np
import pandas as pd
import pickle
import redis
import datetime as dtt
import json
from cma.music.DataQueryClient import DataQueryClient
class rain_hours():
    def __init__(self):
        self.rs = redis.Redis(host='127.0.0.1', port=6379,password="tzqxj58660")
        self.conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="tzqxj58660",db="ZJSZDZDB")
        self.userId = "BEHZ_TZSJ_TZSERVICE" 
        self.pwd = "Liyuan3970!@" 
        self.dataFormat = "json"
    def to_redis(self,name,data):
        self.rs.set(name, pickle.dumps(data))
    def rain_sum(self,delay):
        delay_time = delay+8
        linux = dtt.datetime.now()
        offset = dtt.timedelta(minutes=(-60*8))
        start_delay =  dtt.timedelta(minutes=(-60*delay_time))
        now = (linux+offset).strftime('%Y%m%d%H%M')+"00"
        old = (linux+start_delay).strftime('%Y%m%d%H%M')+"00"
        label = "["+old+","+now+"]"
        client = DataQueryClient(configFile=r"/home/workspace/Data/My_Git/web_met/zdz/common/utils/client.config")
        interfaceId = "statSurfEleInRect"
        params = {
            'dataCode':"SURF_CHN_MUL_MIN",  #SURF_CHN_MUL_HOR
            'elements':"Cnty,Province,Town,Station_levl,Station_Name,City,Station_Id_C,Lat,Lon,Alti",
            'statEles':'SUM_PRE',
            'timeRange':label,
            'minLat':"25",
            'minLon':"115",
            'maxLat':"35",
            'maxLon':"125",
            "statEleValueRanges":"SUM_PRE:(,10000)",
            'limitCnt':"100000000"
        }
        result = client.callAPI_to_serializedStr(self.userId, self.pwd, interfaceId, params, self.dataFormat)
        result_json = json.loads(result)
        clomns =['Cnty','Province','Town','Station_levl','Station_Name','City','Station_Id_C','Lat','Lon','Alti','PRE']
#         clomns = params['elements'].split(",")
        data = pd.DataFrame(result_json['DS'])
        data.columns = clomns
        data = data.astype({'Lat': 'float', 'Lon': 'float','Station_levl':'int','PRE': 'float','Alti':'float'})
        return data

worker = rain_hours()
delay = 24
data = worker.rain_sum(delay)   
name = "rain24"
worker.to_redis(name,data)

URL: http://10.135.90.120:80/music-ws/api?serviceNodeId=NMIC_MUSIC_CMADAAS&method=callAPI_to_serializedStr&userId=BEHZ_TZSJ_TZSERVICE&interfaceId=statSurfEleInRect&language=Python&clientversion=V2.0.0&dataCode=SURF_CHN_MUL_MIN&elements=Cnty,Province,Town,Station_levl,Station_Name,City,Station_Id_C,Lat,Lon,Alti&statEles=SUM_PRE&timeRange=[20230728112900,20230729112900]&minLat=25&minLon=115&maxLat=35&maxLon=125&statEleValueRanges=SUM_PRE:(,10000)&limitCnt=100000000&dataFormat=json&timestamp=1690630144447&nonce=1f917438-2e03-11ee-b6ae-a0369f70ad47&sign=94980B9431EC395FFB1E9AB73EB5AA63
CPU times: user 196 ms, sys: 10.5 ms, total: 206 ms
Wall time: 6.71 s


# 快报绘图

In [2]:
%time
import base64
import json
import os
from io import BytesIO
from math import isnan
import cinrad
import geopandas as gpd
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

# import matplotlib.pyplot as plt
import netCDF4
import numpy as np
# import modin.pandas as pd
import pandas as pd
import shapefile
import xarray as xr
from affine import Affine
from matplotlib.patches import PathPatch
from matplotlib.path import Path
from mpl_toolkits.basemap import Basemap
from ncmaps import Cmaps
from rasterio import features



import xesmf as xe
import redis
from scipy.interpolate import griddata
from scipy.interpolate import interp1d
# import h5netcdf.legacyapi as netCDF4

os.environ["HDF5_USE_FILE_LOCKING"] = 'FALSE'
import datetime as dtt
from datetime import timezone
from datetime import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap

from pylab import *
from matplotlib.font_manager import FontProperties
import pymysql
from pymysql.converters import escape_string
import pickle
import pymssql 
# 查询历史数据的calss

from astropy.convolution import convolve, Gaussian2DKernel, Tophat2DKernel
from astropy.modeling.models import Gaussian2D
## 同步分钟级数据库
from cma.music.DataQueryClient import DataQueryClient
# ftp
import geojsoncontour
from ftplib import FTP
import os
import datetime as dtt
import shutil
import time


%matplotlib inline

from matplotlib import font_manager
# 自定义画图类
class nlcmap(LinearSegmentedColormap):
    """A nonlinear colormap"""

    name = 'nlcmap'

    def __init__(self, cmap, levels):
        self.cmap = cmap
        self.monochrome = self.cmap.monochrome
        self.levels = np.asarray(levels, dtype='float64')
        self._x = self.levels/ self.levels.max()
        self.levmax = self.levels.max()
        self.levmin = self.levels.min()
        self._y = np.linspace(self.levmin, self.levmax, len(self.levels))

    def __call__(self, xi, alpha=1.0, **kw):
        yi = np.interp(xi, self._x, self._y)
        return self.cmap(yi/self.levmax, alpha)

class server_plot():
    def __init__(self,start,end,city,plot_type):
        self.start = start 
        self.end = end
        self.city = city
        self.plot_type = plot_type
        self.rs = redis.Redis(host='127.0.0.1', port=6379,password="tzqxj58660")
        self.conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="tzqxj58660",db="ZJSZDZDB")
        self.userId = "BEHZ_TZSJ_TZSERVICE" 
        self.pwd = "Liyuan3970!@" 
        self.dataFormat = "json"
        self.shp_path = "/home/workspace/Data/My_Git/web_met/static/data/shpfile/xian/"
        self.max = None
        self.min = None
        self.city_info = {
            "taizhou":{
                "static":r"""<table class="layui-table" style="width: 90%;margin-left: 5%;margin-right: 5%;font-size: 5px;">
                    <thead>
                      <tr>
                        <th>  </th>
                        <th>{0[0]}</th>
                        <th>{0[1]}</th>
                        <th>{0[2]}</th>
                        <th>{0[3]}</th>
                        <th>{0[4]}</th>
                        <th>{0[5]}</th>
                        <th>{0[6]}</th>
                        <th>{0[7]}</th>
                        <th>{0[8]}</th>
                        <th>{0[9]}</th>
                        <th>{0[10]}</th>
                      </tr> 
                    </thead>
                    <tbody>
                      <tr>
                        <td>雨量</td>
                        <td>{1[0]}</td>
                        <td>{1[1]}</td>
                        <td>{1[2]}</td>
                        <td>{1[3]}</td>
                        <td>{1[4]}</td>
                        <td>{1[5]}</td>
                        <td>{1[6]}</td>
                        <td>{1[7]}</td>
                        <td>{1[8]}</td>
                        <td>{1[9]}</td>
                        <td>{1[10]}</td>
                      </tr>
                    </tbody>
                  </table>""",
                "data":{
                    "三门":[28.99,121.41],
                    "临海":[28.82,121.15],
                    "仙居":[28.73,120.62],
                    "天台":[29.15,120.97],
                    "椒江":[28.66,121.42],
                    "温岭":[28.38,121.35],
                    "路桥":[28.54,121.41],
                    "黄岩":[28.59,121.04], 
                    "玉环":[28.15,121.23],
                    "平均":None,
                    "最大":None}       
                    },
                "wind":None          
        }
        
    def city_shp(self,data_xr):
        shp_da = self.add_shape_coord_from_data_array(data_xr,self.shp_path+"xian.shp", "country")
        awash_da = shp_da.where(shp_da.country<9, other=np.nan)
        return awash_da
    def transform_from_latlon(self,lat, lon):
        lat = np.asarray(lat)
        lon = np.asarray(lon)
        trans = Affine.translation(lon[0], lat[0])
        scale = Affine.scale(lon[1] - lon[0], lat[1] - lat[0])
        return trans * scale    
    def rasterize(self,shapes, coords, latitude='lat', longitude='lon',fill=np.nan, **kwargs):
        transform = self.transform_from_latlon(coords[latitude], coords[longitude])
        out_shape = (len(coords[latitude]), len(coords[longitude]))
        raster = features.rasterize(shapes, out_shape=out_shape,
                                fill=fill, transform=transform,
                                dtype=float, **kwargs)
        spatial_coords = {latitude: coords[latitude], longitude: coords[longitude]}
        return xr.DataArray(raster, coords=spatial_coords, dims=(latitude, longitude))
    def add_shape_coord_from_data_array(self,xr_da, shp_path, coord_name):   
        shp_gpd = gpd.read_file(shp_path)
        shapes = [(shape, n) for n, shape in enumerate(shp_gpd.geometry)]
        xr_da[coord_name] = self.rasterize(shapes, xr_da.coords, longitude='lon', latitude='lat')
        return xr_da
    def basemask(self,cs, ax, map, shpfile):
        sf = shapefile.Reader(shpfile)
        vertices = []
        codes = []
        for shape_rec in sf.shapeRecords():
            if shape_rec.record[0]:  
                pts = shape_rec.shape.points
                prt = list(shape_rec.shape.parts) + [len(pts)]
                for i in range(len(prt) - 1):
                    for j in range(prt[i], prt[i+1]):
                        vertices.append(map(pts[j][0], pts[j][1]))
                    codes += [Path.MOVETO]
                    codes += [Path.LINETO] * (prt[i+1] - prt[i] -2)
                    codes += [Path.CLOSEPOLY]
                clip = Path(vertices, codes)
                clip = PathPatch(clip, transform = ax.transData)    
        for contour in cs.collections:
            contour.set_clip_path(clip)
    def color_map(self):
        if self.plot_type=="rain":
            start_time = dtt.datetime.strptime(self.start, "%Y-%m-%d %H:%M:%S")
            end_time = dtt.datetime.strptime(self.end, "%Y-%m-%d %H:%M:%S")
            hours = (end_time-start_time).total_seconds()//3600
            if hours >12:
                colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 24降水
                levels = [0,1,10,25,50,100,250,1000]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels)
            elif hours <=12 and hours >6:
                colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 12降水
                levels = [0,1,5,15,30,70,140,250]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels)
            elif hours <=6 and hours >3:
                colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 06降水
                levels = [0,1,4,13,25,60,120,250]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels)
            elif hours <=3 and hours >1:
                colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 03降水
                levels = [0,1,3,10,20,50,70,150]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels)
            elif hours <=1:
                colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 01降水
                levels = [0,1,2,7,15,40,50,100]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels) 
        elif self.plot_type=="wind":
            colorslist = ['#FFFFFF','#CED9FF','#9CFFFF',"#42F217","#FF9E63","#DF16EE","red"]# 风力
            levels = [0,1.6,3.4,5.5,13.9,17.3,32.6,56]
            cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
            cmap_nonlin = nlcmap(cmaps, levels) 
        elif self.plot_type=="tmax":
            level = list(np.linspace(self.min-1, self.max+1, num=14, endpoint=True, retstep=False, dtype=None))
            levels = [round(i,1) for i in level]
            cmap_nonlin = 'Reds'
        elif self.plot_type=="tmin":
            level = list(np.linspace(self.min-1, self.max+1, num=14, endpoint=True, retstep=False, dtype=None))
            levels = [round(i,1) for i in level]
            cmap_nonlin = 'Blues_r'      
        return cmap_nonlin,levels
    def get_rain_data(self):
        start_time = dtt.datetime.strptime(self.start, "%Y-%m-%d %H:%M:%S")
        end_time = dtt.datetime.strptime(self.end, "%Y-%m-%d %H:%M:%S")
        offset = dtt.timedelta(minutes=(-60*8))
        now = (end_time+offset).strftime('%Y%m%d%H%M')+"00"
        old = (start_time+offset).strftime('%Y%m%d%H%M')+"00"
        label = "["+old+","+now+"]"
        client = DataQueryClient(configFile=r"/home/workspace/Data/My_Git/web_met/zdz/common/utils/client.config")
        interfaceId = "statSurfEleInRegion"
        params = {
            'dataCode':"SURF_CHN_MUL_MIN",  #SURF_CHN_MUL_HOR
            'elements':"Cnty,Province,Town,Station_levl,Station_Name,City,Station_Id_C,Lat,Lon,Alti",
            'statEles':'SUM_PRE',
            'timeRange':label,
            'adminCodes':"331000",#330000 浙江省
            "statEleValueRanges":"SUM_PRE:(0,10000)",
            'limitCnt':"100000000"
        }
        result = client.callAPI_to_serializedStr(self.userId, self.pwd, interfaceId, params, self.dataFormat)
        result_json = json.loads(result)
        clomns =['Cnty','Province','Town','Station_levl','Station_Name','City','Station_Id_C','Lat','Lon','Alti','PRE']
        data = pd.DataFrame(result_json['DS'])
        data.columns = clomns
        data = data.astype({'Lat': 'float', 'Lon': 'float','Station_levl':'int','PRE': 'float','Alti':'float'})
        data['value'] = data['PRE']
        rain_mean = data.groupby(['Cnty'])['value'].mean().to_dict()
        rain = sorted(rain_mean.items(),key = lambda x:x[1],reverse = True)
        key_list = []
        value_list = []
        for i in range(len(rain)):
            key = rain[i][0][0:2]
            value = round(rain[i][1],1)
            key_list.append(key)
            value_list.append(str(value))
            if key in self.city_info[self.city]['data'].keys():
                self.city_info[self.city]['data'][key].append(value)
        key_list.append("最大")
        key_list.append("平均")
        value_list.append(str(round(max(data['PRE']),1)))
        value_list.append(str(round(data['PRE'].mean(),1)))
        totle = self.city_info[self.city]['static']
        replace = totle.format(key_list, value_list)
        self.city_info[self.city]['static'] = replace
        self.city_info[self.city]['data']['平均'] = round(data['PRE'].mean(),1)
        self.city_info[self.city]['data']['最大'] = str(round(max(data['PRE']),1))
        return data
    def get_wind_data(self):
        start_time = dtt.datetime.strptime(self.start, "%Y-%m-%d %H:%M:%S")
        end_time = dtt.datetime.strptime(self.end, "%Y-%m-%d %H:%M:%S")
        offset = dtt.timedelta(minutes=(-60*8))
        now = (end_time+offset).strftime('%Y-%m-%d %H:%M:%S')
        old = (start_time+offset).strftime('%Y-%m-%d %H:%M:%S')
        sql = """select max(City) as City,max(Cnty) as Cnty, Station_Id_C , max(Province) as Province,max(Station_levl) as Station_levl,
            max(Station_Name) as Station_Name, max(Town) as Town, max(Alti) as Alti, max(Lat) as Lat,max(Lon) as Lon, max(wind) as wind
            from table_station_hour 
            where Datetime between '{start_times}' and '{end_times}' and wind>0 and City='台州市'
            group by Station_Id_C""" 
        rsql = sql.format(start_times=old,end_times=now)
        data = pd.read_sql(rsql, con=self.conn)
        data['WIN_S_Gust_Max'] = data.apply(lambda x: (x.wind - int(str(int(x.wind))[-3:]))/10000, axis = 1)
        data['WIN_D_Gust_Max'] = data.apply(lambda x: int(str(int(x.wind))[-3:]), axis = 1)
        data['value'] = data['WIN_S_Gust_Max']
        self.city_info[self.city]['data']['平均'] = round(data['WIN_S_Gust_Max'].mean(),1)
        self.city_info[self.city]['data']['最大'] = str(round(max(data['WIN_S_Gust_Max']),1))      
        return data
    def get_temp_data(self):
        start_time = dtt.datetime.strptime(self.start, "%Y-%m-%d %H:%M:%S")
        end_time = dtt.datetime.strptime(self.end, "%Y-%m-%d %H:%M:%S")
        offset = dtt.timedelta(minutes=(-60*8))
        now = (end_time+offset).strftime('%Y%m%d%H%M')+"00"
        old = (start_time+offset).strftime('%Y%m%d%H%M')+"00"
        label = "["+old+","+now+"]"
        client = DataQueryClient(configFile=r"/home/workspace/Data/My_Git/web_met/zdz/common/utils/client.config")
        interfaceId = "statSurfEleInRegion"
        params = {
            'dataCode':"SURF_CHN_MUL_MIN",  #SURF_CHN_MUL_HOR
            'elements':"Cnty,Province,Town,Station_levl,Station_Name,City,Station_Id_C,Lat,Lon,Alti",
            'statEles':'MAX_TEM,MIN_TEM',
            'timeRange':label,
            'adminCodes':"331000",#330000 浙江省
            'eleValueRanges':"TEM:(0,10000)",
            
            'limitCnt':"100000000"
        }
        result = client.callAPI_to_serializedStr(self.userId, self.pwd, interfaceId, params, self.dataFormat)
        result_json = json.loads(result)
        clomns =['Cnty','Province','Town','Station_levl','Station_Name','City','Station_Id_C','Lat','Lon','Alti','tmax','tmin']
        data = pd.DataFrame(result_json['DS'])
        data.columns = clomns
        data = data.astype({'Lat': 'float', 'Lon': 'float','Station_levl':'int','Alti':'float','tmax':'float','tmin':'float'})
        if self.plot_type=="tmax":
            data['value'] = data['tmax']
            self.min = data['value'].min()
            self.max = data['value'].max()
        else:
            data['value'] = data['tmin']
            self.min = data['value'].min()
            self.max = data['value'].max()
        return data
    def decode_uv(self,wspd,wdir):
        deg = 180.0/np.pi
        rad = np.pi/180.0
        u = []
        v = []
        for i in range(len(wspd)):
            u.append(-wspd[i]*np.sin(wdir[i]*rad))
            v.append(-wspd[i]*np.cos(wdir[i]*rad))
        u_np = np.array(u)
        v_np = np.array(v)
        return u_np,v_np
    def decode_xarray(self):
        if self.plot_type=="rain":
            data = self.get_rain_data()
        elif self.plot_type=="wind":
            data = self.get_wind_data()
            self.city_info[self.city]['wind'] = data[data['WIN_S_Gust_Max']>13.8] 
        elif self.plot_type=="tmax" or self.plot_type=="tmin":
            data = self.get_temp_data() 
        lat = np.array(data['Lat'].to_list())
        lon = np.array(data['Lon'].to_list())
        Zi = np.array(data['value'].to_list())
        data_max = max(Zi)
        data_min = min(Zi)
        np.set_printoptions(precision = 2)
        x = np.arange(120.0,122.0,0.05)
        y = np.arange(27.8,29.5,0.05)
        nx0 =len(x)
        ny0 =len(y)
        X, Y = np.meshgrid(x, y)#100*100
        P = np.array([X.flatten(), Y.flatten() ]).transpose()    
        Pi =  np.array([lon, lat ]).transpose()
        Z_linear = griddata(Pi, Zi, P, method = "nearest").reshape([ny0,nx0])
        gauss_kernel = Gaussian2DKernel(0.8)
        smoothed_data_gauss = convolve(Z_linear, gauss_kernel)
        data_xr = xr.DataArray(smoothed_data_gauss, coords=[ y,x], dims=["lat", "lon"])
        return data_xr
    def plot_mark(self,ax,m,awash_da):
        font = font_manager.FontProperties(fname="/home/workspace/Data/My_Git/web_met/static/data/simkai.ttf")
        if self.plot_type=="rain":
            data = self.city_info[self.city]['data']
            for key, value in data.items():
                if type(value)==list:
                    ax.text(value[1], value[0], str(value[2]),fontsize=10,fontproperties=font, color='black',bbox=dict(facecolor='yellow', edgecolor='red'),alpha=0.5)
                    ax.text(value[1]+0.008, value[0]+0.05, key,fontsize=10, weight=5,color='black',alpha=1,fontproperties=font)
        elif self.plot_type=="wind":
            x = np.array(self.city_info[self.city]['wind']['Lon'].to_list())
            y = np.array(self.city_info[self.city]['wind']['Lat'].to_list())
            wspd = np.array(self.city_info[self.city]['wind']['WIN_S_Gust_Max'].to_list())
            wdir = np.array(self.city_info[self.city]['wind']['WIN_D_Gust_Max'].to_list())
            u,v = self.decode_uv(wspd,wdir)
            ax.barbs(x,y,u,v, color='red') 
        elif self.plot_type=="tmin" or self.plot_type=="tmax": 
            len_lat = len(awash_da.lat.data)
            len_lon = len(awash_da.lon.data)
            for i in range(len_lon-1):
                for j in range(len_lat-1):
                    y0 = round(27.8+j*0.05,2)
                    x0 = round(120.0+i*0.05,2)
                    if not isnan(awash_da.data[j,i]):
                        ax.text(x0,y0,str(int(awash_da.data[j,i])),fontsize= 7,fontweight = 800 ,color ="black")
    def plot_img(self):
        data_xr = self.decode_xarray()
        # ##########色标和大小#############################
        cmaps ,levels = self.color_map()
        fig = plt.figure(figsize=[10,10]) 
        ax = fig.add_subplot(111)
        awash_da = self.city_shp(data_xr)
        lat = data_xr.lat
        lon = data_xr.lon
        m = Basemap(llcrnrlon=120.2,
            llcrnrlat=27.8,
            urcrnrlon=122,
            urcrnrlat=29.5,
            resolution = None, 
            projection = 'cyl')
        lons, lats = np.meshgrid(lon, lat)
        cs =m.contourf(lons,lats,data_xr,ax=ax, cmap=cmaps,levels =levels)
        m.readshapefile(self.shp_path + 'xian','xian',color='k',linewidth=1.2)
        plt.axis('off')  
        # 在图上绘制色标
        rect1 = [0.35, 0.25, 0.03, 0.12]        
        ax2 = plt.axes(rect1,frameon='False')
        ax2.set_xticks([])
        ax2.set_yticks([])
        ax2.spines['top'].set_visible(False)
        ax2.spines['bottom'].set_visible(False)
        ax2.spines['left'].set_visible(False)
        ax2.spines['right'].set_visible(False)
        m.colorbar(cs, location='right', size='30%', pad="-100%",ax = ax2)       
        self.basemask(cs, ax, m, self.shp_path+'xian') 
        # 绘制mark
        self.plot_mark(ax,m,awash_da)    
        buffer = BytesIO()
        plt.savefig(buffer,bbox_inches='tight')  
        plot_img = buffer.getvalue()
        imb = base64.b64encode(plot_img) 
        ims = imb.decode()
        imd = "data:image/png;base64,"+ims
#         plt.close()
        return imd


plot_type = "rain"
city = "taizhou"
start = '2023-08-24 20:00:00'
end = '2023-08-25 20:00:00' 
worker = server_plot(start,end,city,plot_type)
# data = worker.plot_rain()
data = worker.plot_img()

# plot_type = "wind"
# worker = server_plot(start,end,city,plot_type)
# worker.get_wind_data()


# plot_type = "tmin"
# worker = server_plot(start,end,city,plot_type)
# # data = worker.get_temp_data()
# img = worker.plot_img()



CPU times: user 5 µs, sys: 3 µs, total: 8 µs
Wall time: 15.3 µs


AttributeError: 'server_plot' object has no attribute 'plot_rain'

17.9

# 数据对接的备份

In [ ]:
         self.city_info = {
            "taizhou":{
                "static":r"""<table class="layui-table" style="width: 90%;margin-left: 5%;margin-right: 5%;font-size: 5px;">
                    <thead>
                      <tr>
                        <th>  </th>
                        <th>{0[0]}</th>
                        <th>{0[1]}</th>
                        <th>{0[2]}</th>
                        <th>{0[3]}</th>
                        <th>{0[4]}</th>
                        <th>{0[5]}</th>
                        <th>{0[6]}</th>
                        <th>{0[7]}</th>
                        <th>{0[8]}</th>
                        <th>{0[9]}</th>
                        <th>{0[10]}</th>
                      </tr> 
                    </thead>
                    <tbody>
                      <tr>
                        <td>雨量</td>
                        <td>{1[0]}</td>
                        <td>{1[1]}</td>
                        <td>{1[2]}</td>
                        <td>{1[3]}</td>
                        <td>{1[4]}</td>
                        <td>{1[5]}</td>
                        <td>{1[6]}</td>
                        <td>{1[7]}</td>
                        <td>{1[8]}</td>
                        <td>{1[9]}</td>
                        <td>{1[10]}</td>
                      </tr>
                    </tbody>
                  </table>""",
                "三门":[28.99,121.46],
                "临海":[28.82,121.15],
                "仙居":[28.69,120.62],
                "天台":[29.01,120.97],
                "椒江":[28.66,121.42],
                "温岭":[28.38,121.35],
                "路桥":[28.54,121.41],
                "黄岩":[28.57,121.04], 
                "玉环":[28.18,121.26],
                "mean":None,
                "max":None
            }
        }
    print(rain)
        for i in rain_mean.items():
            ind = str(key)[0:2]
            print(value)
            if ind in self.city_info[self.city].keys():
                self.city_info[self.city][ind].append(value)

In [ ]:
class station_plot:
    def __init__(self, plot_start_time,plot_end_time,plot_type,color_label,city):
        self.start = plot_start_time
        self.end = plot_end_time
        self.plot_type = plot_type
        self.color_label = color_label
        #self.conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="051219",db="ZJSZDZDB")
        # self.conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="051219",db="ZJSZDZDB")
        self.conn = pymssql.connect("172.21.158.201","down","downx","ZJSZDZDB")
        self.max = None
        self.min = None 
        self.city = city
        self.shp_path = "static/data/shpfile/xian/"
    # 外部函数
    def transform_from_latlon(self,lat, lon):
        lat = np.asarray(lat)
        lon = np.asarray(lon)
        trans = Affine.translation(lon[0], lat[0])
        scale = Affine.scale(lon[1] - lon[0], lat[1] - lat[0])
        return trans * scale    
    def rasterize(self,shapes, coords, latitude='lat', longitude='lon',fill=np.nan, **kwargs):
        transform = self.transform_from_latlon(coords[latitude], coords[longitude])
        out_shape = (len(coords[latitude]), len(coords[longitude]))
        raster = features.rasterize(shapes, out_shape=out_shape,
                                fill=fill, transform=transform,
                                dtype=float, **kwargs)
        spatial_coords = {latitude: coords[latitude], longitude: coords[longitude]}
        return xr.DataArray(raster, coords=spatial_coords, dims=(latitude, longitude))
    def add_shape_coord_from_data_array(self,xr_da, shp_path, coord_name):   
        shp_gpd = gpd.read_file(shp_path)
        shapes = [(shape, n) for n, shape in enumerate(shp_gpd.geometry)]
        xr_da[coord_name] = self.rasterize(shapes, xr_da.coords, longitude='lon', latitude='lat')
        return xr_da
    def basemask(self,cs, ax, map, shpfile):
        sf = shapefile.Reader(shpfile)
        vertices = []
        codes = []
        for shape_rec in sf.shapeRecords():
            if shape_rec.record[0]:  
                pts = shape_rec.shape.points
                prt = list(shape_rec.shape.parts) + [len(pts)]
                for i in range(len(prt) - 1):
                    for j in range(prt[i], prt[i+1]):
                        vertices.append(map(pts[j][0], pts[j][1]))
                    codes += [Path.MOVETO]
                    codes += [Path.LINETO] * (prt[i+1] - prt[i] -2)
                    codes += [Path.CLOSEPOLY]
                clip = Path(vertices, codes)
                clip = PathPatch(clip, transform = ax.transData)    
        for contour in cs.collections:
            contour.set_clip_path(clip)    
    # 内部函数
    def get_sql_data(self):      
        '''sql 获取数据'''
        table = 'Tab_AM_M'
        sqlview = """select ta.IIIII,station.lat,station.lon,min(V)*10 as value
        from {table} as ta inner join TAB_StationInfo as station on ta.IIIII=station.IIiii  
        where (TTime between '{start}' and '{end}' and station.lon>119 and station.lon<122 and station.lat>27.5 and station.lat<29.5 ) group by ta.IIIII,station.lat,station.lon """
        
        sqlrain = """select ta.IIIII,station.lat,station.lon,sum(Ri) as value
        from {table} as ta inner join TAB_StationInfo as station on ta.IIIII=station.IIiii  
        where (TTime between '{start}' and '{end}' and station.lon>119 and station.lon<122 and station.lat>27.5 and station.lat<29.5 ) group by ta.IIIII,station.lat,station.lon """
        
        sqlwind = """select ta.IIIII,station.lat,station.lon,max(fFy) as value
        from {table} as ta inner join TAB_StationInfo as station on ta.IIIII=station.IIiii  
        where (TTime between '{start}' and '{end}' and station.lon>119 and station.lon<122 and station.lat>27.5 and station.lat<29.5 ) group by ta.IIIII,station.lat,station.lon """
        
        sqltmax = """select ta.IIIII,station.lat,station.lon,max(T) as value
        from {table} as ta inner join TAB_StationInfo as station on ta.IIIII=station.IIiii  
        where (TTime between '{start}' and '{end}' and station.lon>119 and station.lon<122 and station.lat>27.5 and station.lat<29.5 and station.ZoomLevel<20 ) group by ta.IIIII,station.lat,station.lon """
        
        sqltmin = """select ta.IIIII,station.lat,station.lon,min(T) as value
        from {table} as ta inner join TAB_StationInfo as station on ta.IIIII=station.IIiii  
        where (TTime between '{start}' and '{end}' and station.lon>119 and station.lon<122 and station.lat>27.5 and station.lat<29.5 and station.ZoomLevel<20) group by ta.IIIII,station.lat,station.lon """
        if self.plot_type=="tmax":
            sql = sqltmax
        elif self.plot_type=="tmin":
            sql = sqltmin
        elif self.plot_type=="rain":
            sql = sqlrain
        elif self.plot_type=="wind":
            sql = sqlwind
        elif self.plot_type=="view":
            sql = sqlview   
        rsql = sql.format(start=self.start,end=self.end,table=table)
        data = pd.read_sql(rsql , con=self.conn)
        drop = data[data['value']!=-9999]
        drop['value'] = drop['value']/10
        lat = np.array(drop['lat'].to_list())
        lon = np.array(drop['lon'].to_list())
        Zi = np.array(drop['value'].to_list())
        data_max = max(Zi)
        data_min = min(Zi)
        self.max = data_max
        self.min = data_min
        # print("测试温度",data_max,data_min)
        np.set_printoptions(precision = 2)
        x = np.arange(120.0,122.0,0.025)
        y = np.arange(27.8,29.5,0.025)
        nx0 =len(x)
        ny0 =len(y)
        X, Y = np.meshgrid(x, y)#100*100
        P = np.array([X.flatten(), Y.flatten() ]).transpose()    
        Pi =  np.array([lon, lat ]).transpose()
        Z_linear = griddata(Pi, Zi, P, method = "nearest").reshape([ny0,nx0])
        gauss_kernel = Gaussian2DKernel(0.3)
        smoothed_data_gauss = convolve(Z_linear, gauss_kernel)
        data_xr = xr.DataArray(smoothed_data_gauss, coords=[ y,x], dims=["lat", "lon"])
        #data_xr = xr.DataArray(Z_linear, coords=[ y,x], dims=["lat", "lon"])
        return data_xr
    def city_shp(self,data_xr):
        if self.city =="taizhou":
            shp_da = self.add_shape_coord_from_data_array(data_xr,self.shp_path+"xian.shp", "country")
            awash_da = shp_da.where(shp_da.country<7, other=np.nan)
        elif self.city =="jiaojiang":
            shp_da = self.add_shape_coord_from_data_array(data_xr, self.shp_path+"xian.shp", "country")
            awash_da = shp_da.where(shp_da.country<7, other=np.nan)
        return awash_da
    def colormap(self):
        '''色标的自定义'''
        plt.rcParams['axes.facecolor']='snow'
        # 降水
        if self.plot_type=="rain":
            if self.color_label =="rain_24hours":
                colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 24降水
                levels = [0,1,10,25,50,100,250,1000]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels)
            elif self.color_label =="rain_12hours":
                colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 12降水
                levels = [0,1,5,15,30,70,140,250]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels)
            elif self.color_label =="rain_06hours":
                colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 06降水
                levels = [0,1,4,13,25,60,120,250]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels)
            elif self.color_label =="rain_03hours":
                colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 03降水
                levels = [0,1,3,10,20,50,70,150]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels)
            elif self.color_label =="rain_01hours":
                colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 01降水
                levels = [0,1,2,7,15,40,50,100]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels)
        elif self.plot_type=="tmax" or self.plot_type=="tmin":
            if self.color_label =="tmax_normal" or self.color_label =="tmin_normal":
                #colorslist = ['#1F1FFF',"#3B3BFF","#5757FF","#7272FF","#8F8FFF","#ABABFF","#C7C7FF","#E3E3FF","#FDFC8B","#F8E08B","#F3C36F","#EFA76E","#EC8A51","#F31717"]# 气温
                #colornum = len(colorslist)
                level = list(np.linspace(self.min-1, self.max+1, num=14, endpoint=True, retstep=False, dtype=None))
                levels = [round(i,1) for i in level]
                cmap_nonlin = 'seismic'#'coolwarm'#
            elif self.color_label =="tmax_high":
                level = list(np.linspace(self.min-1, self.max+1, num=14, endpoint=True, retstep=False, dtype=None))
                levels = [round(i,1) for i in level]
                cmap_nonlin = 'Reds'
            elif self.color_label =="tmin_low":
                level = list(np.linspace(self.min-1, self.max+1, num=14, endpoint=True, retstep=False, dtype=None))
                levels = [round(i,1) for i in level]
                cmap_nonlin = 'Blues_r'
        elif self.plot_type=="wind":
            if self.color_label =="wind_normal":
                colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 风力
                levels = [0,1,10,15,25,50,100,250]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels) 
            elif self.color_label =="wind_other":
                colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 风力其他
                levels = [0,1,10,15,25,50,100,250]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels)
        elif self.plot_type=="view":
            if self.color_label =="view_normal":
                colorslist = ['#A93434','#FF9600','#FFFD37',"#55FF37","#ABF3D3","#5EC6EB","#B1F1EF"]# 能见度
                levels = [0,50,200,500,1000,1500,3000,20000]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels) 
            elif self.color_label =="view_other":
                colorslist = ['#FFFFFF','#FF9600','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 能见度
                levels = [0,1,10,15,25,50,100,250]
                cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
                cmap_nonlin = nlcmap(cmaps, levels)             
        return cmap_nonlin ,levels
    def plot_mark(self):
        pass
    def plot_label(self):
        pass
    def plot_img(self):
        data_xr = self.get_sql_data()
        # 平滑
        #data_xr = scipy.ndimage.zoom(data_xr, 3)
        # ##########色标和大小#############################
        cmaps ,levels = self.colormap()
        fig = plt.figure(figsize=[10,10]) 
        ax = fig.add_subplot(111)
        awash_da = self.city_shp(data_xr)
        lat = data_xr.lat
        lon = data_xr.lon
        m = Basemap(llcrnrlon=120.2,
            llcrnrlat=27.8,
            urcrnrlon=122,
            urcrnrlat=29.5,
            resolution = None, 
            projection = 'cyl')
        lons, lats = np.meshgrid(lon, lat)
        cs =m.contourf(lons,lats,data_xr,ax=ax, cmap=cmaps,levels =levels,add_labels=True)
        ##########标题#############################
        # font = FontProperties(fname="simkai.ttf", size=14)
        # 为matplotlib中文无法显示设置字体
        #plt.rcParams['font.sans-serif'] = 'SimHei' # 黑体
        # label  = "测试绘图"
        # plt.text(120.2,29.4, label,fontsize=15, fontproperties=font)
        ##########标题#############################
        m.readshapefile(self.shp_path + 'xian','xian',color='k',linewidth=1.2)
        plt.axis('off')
        # 在图上绘制色标
        rect1 = [0.35, 0.25, 0.03, 0.12]         
        ax2 = plt.axes(rect1,frameon='False')
        ax2.set_xticks([])
        ax2.set_yticks([])
        ax2.spines['top'].set_visible(False)
        ax2.spines['bottom'].set_visible(False)
        ax2.spines['left'].set_visible(False)
        ax2.spines['right'].set_visible(False)
        m.colorbar(cs, location='right', size='30%', pad="-100%",ax = ax2)
        self.basemask(cs, ax, m, self.shp_path+'xian') 
        buffer = BytesIO()
        plt.savefig(buffer,bbox_inches='tight')  
        plot_img = buffer.getvalue()
        imb = base64.b64encode(plot_img) 
        ims = imb.decode()
        imd = "data:image/png;base64,"+ims
        return imd